In [1]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject
from resample_tools import warp_image
from rasterio.warp import Resampling
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

#### User needs to modify this line to correspond to their local data, 
#### which can be downloaded from Hydroshare:
#### https://www.hydroshare.org/resource/8b76906c4b604c458fbcb5ea7c8c0be7
LOC_DATA_DIR = "./data/"

STATS_DIR = f"{LOC_DATA_DIR}images_for_sacramento_stats/"

# Sentinel-2, NWM-CNN, HAND, global surface water, and pre-event-water
We have five files to deal with:
  - `CA_S1_20230106_20230114_masked_clip_buffer`. This file is the "ground truth" that we'll use to compare the model performance. 
  - `GSW_binary_gt30`. This is the global surface water that we want to ignore in our calculations.
  - `NWM-HAND_180201_20230106-13_max_depth`. This is the HAND flood map we are using as a baseline for our comparison.
  - `s2_max_extent_20221101_20221215`. This is a pre-event surface water extent map.
  - `NWM-CNN_20230110_clipped_18020104`. This is our model flood map.

# Make CA_S1 match the NWM-CNN extent, projection and resolution
In order to calculate an RMSE we need to have the rasters match.  
The CA_S1 map is binary, so we'll want to resample it such that the resulting map has values between 0 and 100, just like the V2 output. 

In [2]:
# Paths for your files
binary_raster_path = f"{STATS_DIR}CA_S1_20230106_20230114_masked_clip_buffer.tif"
target_raster_path = f"{STATS_DIR}NWM-CNN_20230106_20230114.tif"
output_path = f"{STATS_DIR}CA_S1.tif"

# Load target raster's properties
with rasterio.open(target_raster_path) as target:
    target_transform = target.transform
    target_crs = target.crs
    target_shape = target.shape

# Load binary raster
with rasterio.open(binary_raster_path) as src:

    # Calculate the transformation between source and target
    transform, width, height = calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds,
        dst_width=target_shape[1], dst_height=target_shape[0]
    )

    # Create a new array with target dimensions to store results
    resampled_array = np.empty(shape=target_shape, dtype=np.float32)

    # Resample the binary raster using the average technique
    reproject(
        source=rasterio.band(src, 1),
        destination=resampled_array,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=target_transform,
        dst_crs=target_crs,
        resampling=Resampling.average
    )

    # Adjust values to represent percentage
    resampled_array = resampled_array * 100  # assuming original True values are 1, else adjust this factor

    # Save the result to a new raster
    with rasterio.open(output_path, 'w', driver='GTiff', height=target_shape[0], 
                       width=target_shape[1], count=1, dtype=np.float32, 
                       crs=target_crs, transform=target_transform) as dest:
        dest.write(resampled_array, 1)


# Make GSW match the extent and resolution of the NWM-CNN and CA_S1
This map is binary, and we want a resulting binary map at the end

In [3]:
def resample_binary_to_match(source_path, target_path, output_path):
    """Resample a binary raster to match the extent, projection, and resolution of another raster."""
    
    with rasterio.open(source_path) as src, rasterio.open(target_path) as target:
        # Copy and update the metadata from the target
        metadata = target.meta.copy()
        
        # Update data type to be int16 for the binary output
        metadata['dtype'] = 'int16'
        
        # Create an empty array with the shape of the target raster
        destination = np.empty_like(target.read(1), dtype='int16')

        # Perform the resampling
        reproject(
            source=src.read(1),
            destination=destination,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=target.transform,
            dst_crs=target.crs,
            resampling=Resampling.average
        )

        # Threshold the resampled data to produce a binary raster
        # You can adjust the threshold if needed
        destination[destination >= 0.5] = 1
        destination[destination < 0.5] = 0

        # Save the result to a new file
        with rasterio.open(output_path, 'w', **metadata) as dest:
            dest.write(destination, 1)

# Example usage:
source_binary_path = f"{STATS_DIR}GSW_binary_gt30.tif"
target_raster_path = f"{STATS_DIR}NWM-CNN_20230106_20230114.tif"
output_path = f"{STATS_DIR}GSW.tif"
resample_binary_to_match(source_binary_path, target_raster_path, output_path)
source_binary_path = f"{STATS_DIR}s2_max_extent_20221101_20221215.tif"
target_raster_path = f"{STATS_DIR}NWM-CNN_20230106_20230114.tif"
output_path = f"{STATS_DIR}s2_pre_event.tif"
resample_binary_to_match(source_binary_path, target_raster_path, output_path)

# Resample HAND
Raster called "HAND_180201_20230106-13_max_depth.tif" which has values from 0 to inf, and we need to make it match in resolution and extent a raster called "V2_20230110_clipped_18020104.tif", which has pixel values from 0 to 100. We need the raster "HAND_180201_20230106-13_max_depth.tif" to be resampled such that any pixel value above zero counts towards a percentage of the resampled pixel, such that the final pixel value represents the percentage of original pixels above zero that would be found within the final pixel extent

In [4]:
def resample_to_match(source_path, target_path, output_path):
    """Resample source raster to match resolution and extent of the target raster."""
    
    # Open the target raster to get its transform and dimensions
    with rasterio.open(target_path) as target:
        target_transform = target.transform
        target_crs = target.crs
        target_width = target.width
        target_height = target.height
    
    # Open the source raster, treat no data values as zeros, and binarize
    with rasterio.open(source_path) as src:
        nodata_val = src.nodata  # get the no data value
        src_data = src.read(1)
        
        # Treat nodata values as zeros
        src_data[np.isnan(src_data)] = 0  # If nodata is represented as NaN
        if nodata_val is not None:
            src_data[src_data == nodata_val] = 0
        
        binary_data = np.where(src_data > 0, 1, 0).astype(np.float32)
        
        # Create an array for the resampled data
        resampled_data = np.empty((target_height, target_width), dtype=np.float32)
        
        # Resample
        reproject(
            source=binary_data,
            destination=resampled_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=target_transform,
            dst_crs=target_crs,
            resampling=Resampling.average
        )
        
        # Convert fraction to percentage
        resampled_data *= 100

    # Write out the resampled data
    with rasterio.open(output_path, 'w', driver='GTiff', width=target_width, height=target_height, 
                       count=1, dtype=np.float32, crs=target_crs, transform=target_transform) as dst:
        dst.write(resampled_data, 1)

    return output_path

source_raster = f"{STATS_DIR}NWM-HAND_180201_20230106-13_max_depth_clip.tif"
target_raster = f"{STATS_DIR}NWM-CNN_20230106_20230114.tif"
output_path = f"{STATS_DIR}NWM-HAND.tif"
output = resample_to_match(source_raster, target_raster, output_path)
print(f"Resampled raster saved to: {output}")


Resampled raster saved to: ./data/images_for_sacramento_stats/NWM-HAND.tif


# Binary metrics
#### Now we have to go through a somewhat different method to resample the data
#### --------------------------------------------------------------------------

### Warp CA_S1 to match NWM-CNN

In [5]:
in_file = f"{STATS_DIR}CA_S1_20230106_20230114_masked.tif"
in_profile = rasterio.open(in_file).profile
target_file = f"{STATS_DIR}NWM-CNN_20230111_clipped_18020104.tif"
out_profile = rasterio.open(target_file).profile
out_profile['dtype'] = "float32"
north, west, south, east = 39.7770301587301631, -122.5135134490238613, 38.6947335675675674, -121.6148633188720112, 
bbox = [west, south, east, north]
resolution_m = 250
target_res = resolution_m / 111000  # 250m -> degrees

warped_tif = warp_image(
    tif_list=[in_file],
    dst_bounds=bbox,
    dst_crs=4269,
    res=target_res,
    resampling=Resampling.average,
    src_nodata=0,
    dst_nodata=0,
    profile=out_profile,
    deflate_compression=False,
    output_dir=f"{STATS_DIR}",
)
print(f"This should have made the file: {STATS_DIR}CA_S1_20230106_20230114_masked_resampled250mX100_18020104.tif")

This should have made the file: ./data/images_for_sacramento_stats/CA_S1_20230106_20230114_masked_resampled250mX100_18020104.tif


### Warp HAND to match CA_S1

In [6]:
in_file = f"{STATS_DIR}NWM-HAND_180201_20230106-13_max_depth_binary100_clip.tif"
in_profile = rasterio.open(in_file).profile
target_file = f"{STATS_DIR}CA_S1_20230106_20230114_masked_resampled250mX100_18020104.tif"
out_profile = rasterio.open(target_file).profile
out_profile['dtype'] = "float32"
north, west, south, east = 39.7770301587301631, -122.5135134490238613, 38.6947335675675674, -121.6148633188720112, 
bbox = [west, south, east, north]
resolution_m = 250
target_res = resolution_m / 111000  # 250m -> degrees

warped_tif = warp_image(
    tif_list=[in_file],
    dst_bounds=bbox,
    dst_crs=4269,
    res=target_res,
    resampling=Resampling.max,
    src_nodata=0,
    dst_nodata=0,
    profile=out_profile,
    deflate_compression=False,
    output_dir=f"{STATS_DIR}",
)
print(f"This should have made the file {STATS_DIR}NWM-HAND_180201_20230106-13_max_depth_binary100_clip.nodata0_max_481x399_res0.0023x0.0023.tif")

This should have made the file ./data/images_for_sacramento_stats/NWM-HAND_180201_20230106-13_max_depth_binary100_clip.nodata0_max_481x399_res0.0023x0.0023.tif
